In [15]:
import numpy as np
print(np.__version__)
import traitlets
print(traitlets.__version__)
import scipy.io
import matplotlib.pyplot as plt
import pyedflib
from pyedflib import highlevel

from biosppy.signals.ecg import correct_rpeaks, hamilton_segmenter
from scipy.signal import medfilt
from scipy.interpolate import splev, splrep, interp1d
from scipy.signal import resample
import biosppy.signals.tools as st
import os
import wfdb
import neurokit2 as nk
#import neurokit2 as nk
import joblib
import seaborn as sns
sns.set_theme()


1.21.3
5.1.1


In [16]:
def get_patient_list(base_dir):
    patient_list = []
    for filename in os.listdir(base_dir):
        if filename.endswith('.edf'):
            patient_list.append(filename[:-4])
    return(patient_list)

def get_ecg(base_dir, patient_id):
    file_name = base_dir + '/' + patient_id + '.edf'
    f, signal_header, header = highlevel.read_edf(file_name)
    ecg_channel_id = next((i for i, item in enumerate(signal_header) if item["label"] == "ECG1-ECG2"), None)
    if ecg_channel_id == None:
        return([], None)
    ecg = f[ecg_channel_id]
    fs = signal_header[ecg_channel_id]['sample_rate']
    return(ecg, fs)

def get_hypnogram(base_dir, patient_id):
    file_name = base_dir + '/hyp' + patient_id + '.mat'
    mat = scipy.io.loadmat(file_name)
    stages = mat['hyp'][:,0]
    times = mat['hyp'][:,1]
    return(stages, times)

def complexe_filtering(rri_signal, rri_tm):
    omega = 5
    lb = .4
    ub = 2
    smin, idx = np.min(rri_signal), np.argmin(rri_signal)

    if smin < lb:
        if (idx == 0) or idx == (len(rri_signal)-1):
            rri_signal = np.delete(rri_signal, idx)
            rri_tm = np.delete(rri_tm, idx)
            return(complexe_filtering(rri_signal, rri_tm))

        idx_beg = np.max((0,idx - omega))
        idx_end = np.min((len(rri_signal), idx + omega + 1))
        window = rri_signal[idx_beg:idx_end+1]
        med = np.median(window)
        
        d1 = np.abs(smin + np.min((rri_signal[idx - 1], rri_signal[idx + 1])) - med)
        d2 = 2 * np.abs( (smin + np.max((rri_signal[idx - 1], rri_signal[idx + 1])))/2 - med)
        if d1 < d2:
            rri_signal = np.delete(rri_signal, idx)
            rri_tm = np.delete(rri_tm, idx)
        if d1 >= d2:
            rri_signal[idx] = (rri_signal[idx-1] + rri_signal[idx + 1])/2
        return(complexe_filtering(rri_signal, rri_tm))
    
    smax, idx = np.max(rri_signal), np.argmax(rri_signal)
    if smax <= ub:
        return(rri_signal, rri_tm)

    idx_beg = np.max((0,idx - omega))
    idx_end = np.min((len(rri_signal), idx + omega + 1))
    window = rri_signal[idx_beg:idx_end+1]
    med = np.median(window)
    k = np.max([np.floor(smax/med), 2])
    if idx == 0 or idx == len(rri_signal):
            rri_signal = rri_signal[idx] = rri_signal[idx] / k
            return(complexe_filtering(rri_signal, rri_tm))

    d1 = k * np.abs(smax/k - med)
    d2 = 2 * np.abs( (smax + np.min((rri_signal[idx - 1], rri_signal[idx + 1])))/2 - med)
    if d1 < d2:
        rri_signal[idx] = rri_signal[idx] / k
    if d1 >= d2:
        rri_signal[idx] = (rri_signal[idx-1] + rri_signal[idx + 1])/2
        
    return(complexe_filtering(rri_signal, rri_tm))

In [12]:
def main_worker(base_dir, name, ecg_signal, fs, rri_filtering='median'):
    print("processing %s!" % name)
    stages, times = get_hypnogram(base_dir, name)
    times = list(times)
    X = []
    y = [] #Sleep stage
    #ecg = []
    groups = []
    sizee = 60
    sample = fs * sizee
    ir = 3 # interpolate interval
    before = 2
    after = 2
    hr_min = 20
    hr_max = 300
    tm = np.arange(0, (before + 1 + after) * sizee, step=1 / float(ir))
    for j in range(int(len(ecg_signal) / float(sample))):
        if (j < before) or ((j + 1 + after) > (len(ecg_signal) / float(sample))) or (int(j * sizee) not in times):
            continue
        signal = ecg_signal[int((j - before) * sample):int((j + 1 + after) * sample)]
        signal, _, _ = st.filter_signal(signal, ftype='FIR', band='bandpass', order=int(0.3 * fs),
                                        frequency=[3, 45], sampling_rate=fs) # Filtering the ecg signal to remove noise
        # R peaks
        rpeaks, = hamilton_segmenter(signal, sampling_rate=fs) # Extract R-peaks
        rpeaks, = correct_rpeaks(signal, rpeaks=rpeaks, sampling_rate=fs, tol=0.1)
        if (len(rpeaks) * 60 / (1 + after + before) / sizee < 40) or (len(rpeaks) * 60 / (1 + after + before) / sizee > 200):  # Remove abnormal R peaks signal
            continue
        # Extract RRI, Ampl signal
        rri_tm, rri_signal = rpeaks[1:] / float(fs), np.diff(rpeaks) / float(fs)
        if rri_filtering == 'median':
            rri_signal = medfilt(rri_signal, kernel_size=9)
        if rri_filtering == 'complexe':
            rri_signal, rri_tm = complexe_filtering(rri_signal, rri_tm)
        if rri_filtering == 'complexe+med':
            rri_signal = medfilt(rri_signal, kernel_size=7)
            rri_signal, rri_tm = complexe_filtering(rri_signal, rri_tm)
            
        ampl_tm, ampl_signal = rpeaks / float(fs), signal[rpeaks]
        hr = 60 / rri_signal
        rpeaks, info = nk.ecg_peaks(signal, sampling_rate=fs)
        ecg_rate = nk.ecg_rate(rpeaks, sampling_rate=fs, desired_length=len(signal))
        edr_temp = nk.ecg_rsp(ecg_rate, sampling_rate=fs)
        # Remove physiologically impossible HR signal
        if np.all(np.logical_and(hr >= hr_min, hr <= hr_max)):
            # Save extracted signal
            inter_mode = 'custom'
            if inter_mode == 'base':
                rri_interp_signal = splev(tm, splrep(rri_tm, rri_signal, k=3), ext=3) 
                ampl_interp_signal = splev(tm, splrep(ampl_tm, ampl_signal, k=3), ext=0)
            if inter_mode == 'custom':
                inter = interp1d(rri_tm, rri_signal, kind='linear', bounds_error=False, fill_value=(rri_signal[0], rri_signal[-1]))
                rri_interp_signal = inter(tm)
                inter = interp1d(ampl_tm, ampl_signal, kind='linear', bounds_error=False, fill_value=(ampl_signal[0], ampl_signal[-1]))
                ampl_interp_signal = inter(tm)
            if j == 150:
                plt.plot(rri_interp_signal)
                plt.show()
            edr_interp_signal = resample(edr_temp, (1 + after + before) * sizee * 3)
            #ecg.append([signal, np.arange(int((j - before) * sample), int((j + 1 + after) * sample))])
            X.append([rri_interp_signal, ampl_interp_signal, edr_interp_signal])
            groups.append([name])
            stage = stages[times.index(int(j * sizee))]
            y.append(stage)
    X = np.array(X, dtype="float32").transpose((0, 2, 1)) # convert to numpy format
    return X, y, groups

In [ ]:
title = 'cap_data.pkl'
base_dir = r'C:\thomas\python\cap\data'
patient_list = get_patient_list(base_dir)

X = []
y = []
groups = []
count_signal = 0
count_jump = 0

# 4, 8, 
for patient_id in patient_list:
    print(patient_id)
    ecg, fs = get_ecg(base_dir, patient_id)
    if ecg == []:
        print('Pas de ECG propre')
        count_jump += 1
        continue
    X_loc, y_loc, groups_loc = main_worker(base_dir=base_dir, name=patient_id, ecg_signal=ecg, fs=fs, rri_filtering='median')
    X.extend(X_loc)
    y.extend(y_loc)
    groups.extend(groups_loc)
    count_signal += 1

apnea_ecg = dict(X = X, y = y, groups = groups)
import joblib 
with open(os.path.join('.', title), "wb") as f:
    joblib.dump(apnea_ecg, f, protocol=2)